In [1]:
from backtesting import Backtest, Strategy
from backtesting.test import GOOG
from backtesting.lib import crossover
import seaborn as sns
#import backtesting
#backtesting.set_bokeh_output(notebook=False)
import pandas as pd
import polars as pl
import numpy as np

/home/themanuello/.cache/pypoetry/virtualenvs/financialpython-cSpIym4p-py3.10/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/themanuello/.cache/pypoetry/virtualenvs/financialpython-cSpIym4p-py3.10/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [2]:

def labelize_output(df: pd.DataFrame, target:list, wrt:str='Close', threshold:float=0.01, risk_reward_ratio:float=0.8) -> pd.DataFrame:
    
    df_ = df.copy()
    positive_criterion = 1 + threshold/risk_reward_ratio
    negative_criterion = 1 - threshold
     
    for column in target:
        conditions = [
            (df_[column] >= df_[wrt] * positive_criterion),
            (df_[column] <= df_[wrt] * negative_criterion)]
        choices = [2, 1]
        df_[f"target_{column}"] = np.select(conditions, choices, default=0)
        #df.drop(column,1,inplace=True)
    return df_

def TFs_to_predict(df:pd.DataFrame, apply_on:str, TimeFrame:str, howmanyTFs:int) -> pd.DataFrame:

    #apply_one = any('Open','Close','High','Low')
    for i in range(1, howmanyTFs+1):
        df[f"{TimeFrame}{apply_on}{i}"] = df["Close"].shift(-i)
    
    return df
    


In [11]:
df = pl.read_csv('data/data.csv').to_pandas()
df = df.drop(columns=["label", "timestamp", "turnover","date", "lavel","TP","SL"], errors="ignore")
df.columns = ["Open","High","Low","Close","Volume","signal"]
df[20:]

,Open,High,Low,Close,Volume,signal
20,1201.45,1204.70,1201.40,1202.05,15107.72,0.0
21,1202.05,1205.05,1201.35,1201.40,18256.84,0.0
22,1201.40,1202.95,1200.65,1202.60,6300.14,0.0
23,1202.60,1202.95,1200.40,1201.05,7668.91,0.0
24,1201.05,1202.80,1199.25,1200.60,12027.64,0.0
...,...,...,...,...,...,...
13674,3401.19,3429.16,3386.95,3427.09,99697.78,1.0
13675,3427.09,3452.00,3417.18,3445.61,40659.27,0.0
13676,3445.61,3496.00,3444.26,3456.38,96795.94,0.0
13677,3456.38,3489.44,3453.73,3481.10,52953.16,0.0


# CROSSOVER STRATEGY

In [13]:
def optim_func(series):
    
    if series["# Trades"] < 10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]



class RsiOscillator(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    def init(self):
        # I = indicator
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        self.ema7 = self.I(talib.EMA, self.data.Close, 7)
        self.ema30 = self.I(talib.EMA, self.data.Close, 30)
        self.ema60 = self.I(talib.EMA, self.data.Close, 60)
        self.ema200 = self.I(talib.EMA, self.data.Close, 200)

    def next(self):
        
        if crossover(self.rsi, self.upper_bound) or crossover(self.ema200, self.ema7) :
            self.position.close()
        elif crossover(self.lower_bound, self.rsi) or crossover(self.ema7, self.ema200):
            self.buy()
            

def generate_next_open(open):
    return pd.DataFrame(open).shift(1)

    
class SignalStrategy(Strategy):

    n_days=2

    def init(self):
        
        self.is_long = False
        self.days_passed = 0
        self.profit = 0
        self.entry_price = 0
        self.tp_perc = 1.02
        self.sl_perc = .99

    def next(self):
        current_signal = self.data.signal[-1]
        current_close = self.data.Close[-1]

        if current_signal == 1:
            if not self.position:
                    print(f"{current_close}")
                    #self.buy()
                    self.buy(tp=self.tp_perc*current_close, sl=self.sl_perc*current_close)
            
        if self.position:
            self.days_passed += 1/24

        if self.days_passed >= self.n_days:
                self.position.close()
                self.days_passed = 0

bt = Backtest(df, SignalStrategy, cash = 10_000, commission=0.01)

#bt = Backtest(df, RsiOscillator, cash = 10_000)
#stats = bt.optimize(
#    upper_bound = range(55,85,5),
#    lower_bound = range(10,45,5),
#    rsi_window = range(10, 30, 2),
#    maximize= "Sharpe Ratio"
#)

stats = bt.run()
bt.plot()   

/tmp/ipykernel_12712/2817853367.py:66: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, SignalStrategy, cash = 10_000, commission=0.01)


1223.85
1268.3
1341.55
1388.7
1384.65
1419.55
1452.35
1561.25
1569.15
1559.97
1591.66
1641.09
1561.76
1585.58
1573.9
1571.88
1617.56
1637.96
1672.6
1637.23
1661.44
1630.59
1555.18
1579.98
1628.18
1665.9
1675.99
1672.64
1637.93
1637.01
1620.89
1412.72
1384.13
1400.83
1426.01
1416.06
1455.3
1514.21
1512.0
1540.23
1563.64
1581.81
1681.71
1734.99
1655.64
1664.72
1711.38
1740.52
1772.0
1775.26
1792.8
1757.45
1757.2
1750.28
1745.01
1750.49
1718.06
1734.61
1784.16
1792.5
1792.52
1778.3
1828.68
1865.11
1860.53
1880.76
1909.75
2007.05
2097.52
1939.18
1876.19
1914.71
1863.89
1898.86
1828.08
1872.8
1904.03
1945.75
1841.87
1833.64
1852.29
1841.44
1732.35
1685.36
1718.15
1717.0
1731.74
1790.99
1842.8
1871.55
1870.7
1860.48
1844.05
1917.71
1886.25
1932.41
1618.1
1662.13
1603.19
1651.63
1645.0
1551.68
1577.05
1673.08
1582.84
1635.01
1707.48
1776.23
1799.05
1787.0
1797.5
1821.59
1855.65
1875.18
1868.19
1858.98
1913.72
1993.35
2020.44
2060.84
2051.87
2044.39
2034.4
2080.06
2050.42
2100.16
2149.51
2169.

GridPlot(id='p2222', ...)

Opening in existing browser session.


In [11]:
stats

Start                                     0.0
End                                     569.0
Duration                                569.0
Exposure Time [%]                   23.508772
Equity Final [$]                   34576.9619
Equity Peak [$]                    34576.9619
Return [%]                         245.769619
Buy & Hold Return [%]              189.070465
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                134.0
Win Rate [%]                            100.0
Best Trade [%]                       0.990099
Worst Trade [%]                      0.990099
Avg. Trade [%]                    

In [ ]:
df[599:].groupby('signal').count()